In [1]:
import numpy as np
import pandas as pd
import json
import gensim
from konlpy.tag import Kkma, Twitter
from gensim import corpora

In [12]:
with open('con_A_part.json', 'r') as fp:
    con_A_part = json.load(fp)
with open('pro_A_part.json', 'r') as fp:
    pro_A_part = json.load(fp)
#with open('all_A_part.json', 'r') as fp:
    #all_A_part = json.load(fp)

In [2]:
with open('con_Q_part.json', 'r') as fp:
    con_Q_part = json.load(fp)
with open('pro_Q_part.json', 'r') as fp:
    pro_Q_part = json.load(fp)
#with open('all_Q_part.json', 'r') as fp:
    #all_Q_part = json.load(fp)

In [11]:
with open('con_A_part_vec.json', 'r') as fp:
    con_A_part_vec = json.load(fp)
with open('pro_A_part_vec.json', 'r') as fp:
    pro_A_part_vec = json.load(fp)
#with open('all_A_part_vec.json', 'r') as fp:
    #all_A_part_vec = json.load(fp)

In [4]:
with open('A_estimate_con.json', 'r') as fp:
    A_estimate_con = json.load(fp)
with open('A_estimate_pro.json', 'r') as fp:
    A_estimate_pro = json.load(fp)
#with open('A_estimate_all.json', 'r') as fp:
#    A_estimate_all = json.load(fp)

In [5]:
word2vecmodel = gensim.models.Word2Vec.load('./word2vecmodel')
ldamodel =  gensim.models.ldamodel.LdaModel.load('./ldamodel')
dictionary = corpora.Dictionary.load('ldamodel.id2word')
Tw = Twitter()

# Part를 리턴 해주는 함수

In [6]:
def return_part(q):
    nouns = Tw.nouns(q)
    
    topics = ldamodel.get_document_topics(dictionary.doc2bow(nouns))

    max_prob = round(topics[0][1], 5)
    temp_prob = round(topics[0][1], 5)
    max_part = topics[0][0]
    
    check = 0

    for i in range(1,len(topics)):
        if round(topics[i][1], 5) != temp_prob:
            check = 1
            break
    
    if check == 0:
        return(str(30))

    if check == 1:
        for part,prob in topics:
            if round(prob,5) > max_prob:
                max_prob = round(prob,5)
                max_part = part
    
        return str(max_part)

# 질문을 벡터화해서 리턴 해주는 함수

In [7]:
def return_vector(q):
    
    morphs = Tw.morphs(q)     
    nouns = Tw.nouns(q)
    doc2bow = [dictionary.doc2bow(nouns)]
    
    ldalist = np.zeros(30)
    
    for pair in ldamodel.get_document_topics(doc2bow)[0]:
        ldalist[pair[0]] = pair[1]
    
    k = []
    
    for word in morphs:
        try:
            k.append(word2vecmodel.wv[word])            
        except:
            k.append(np.zeros(200))
    
    k = np.array(k)
    
    vec = np.zeros(200)

    for i in range(len(vec)):
        vec[i] = (np.mean(k[:,i]))
    
    return np.append(vec,ldalist)

# 예상 답을 리턴 해주는 함수 각각 (진보, 보수, 전체)

In [22]:
def return_pro(q):
    if q in hello :
        return hello[q]
    
    part = return_part(q)
    
    vec = return_vector(q)
    
    ex_A = np.dot(vec, A_estimate_pro[part])
    
    min_dis = 1000
    index = 0
    for j in range(len(pro_A_part_vec[part])):
        dis = np.linalg.norm((pro_A_part_vec[part][j]-ex_A))
        if dis < min_dis:
            min_dis = dis
            index = j

    return pro_A_part[part][index]


def return_con(q):
    if q in hello :
        return hello[q]
    
    part = return_part(q)
    vec = return_vector(q)
        
    ex_A = np.dot(vec, A_estimate_con[part])
    
    min_dis = 1000
    index = 0
    for j in range(len(con_A_part_vec[part])):
        dis = np.linalg.norm((con_A_part_vec[part][j]-ex_A))
        if dis < min_dis:
            min_dis = dis
            index = j
    
    return con_A_part[part][index]

def return_all(q):
    if q in hello :
        return hello[q]
    
    part = return_part(q)
    vec = return_vector(q)
        
    ex_A = np.dot(vec, A_estimate_all[part])
    
    min_dis = 1000
    index = 0
    for j in range(len(all_A_part_vec[part])):
        dis = np.linalg.norm((all_A_part_vec[part][j]-ex_A))
        if dis < min_dis:
            min_dis = dis
            index = j
    return all_A_part[part][index]

In [9]:
hello = {'안녕':'반말하지 마십시오.',
         '안녕?':'반말하지 마십시오',
         '안녕하세요': '안녕하세요.',
         '안녕 하세요':'안녕하세요',
         '안녕하세요 위원님': '안녕하세요 위원님.',
         '안녕하세요 의원님':'안녕하세요 의원님.',
        '안녕하세요위원님':'안녕하세요위원님.',
         '안녕하세요의원님': '안녕하세요의원님',
         '안녕히계세요': '아직 회의 안 끝났습니다.',
        '안녕히 계세요': '아직 회의 안 끝났습니다.'}

In [23]:
for i in range(len(con_Q_part['0'])):
    print('Q -> ',con_Q_part['0'][i])
    print('Original A -> ', con_A_part['0'][i])
    
    print('ex A ->',return_con(con_Q_part['0'][i]))
    print('------------------------------------------------------------------------------------')
    if i ==30 :
        break
    


Q ->  아니 그런데 사업은 그렇게 해도 노무현 정부 때 했어요.
Original A ->  노무현 정부 때 다 계획이 수립됐기 때문에
ex A -> 노무현 정부 때 다 계획이 수립됐기 때문에
------------------------------------------------------------------------------------
Q ->  아니 한마디만 하시면 되지 않나요?
Original A ->  그것 하고 나서 그다음에 얘기를 하시면
ex A -> 그것 하고 나서 그다음에 얘기를 하시면
------------------------------------------------------------------------------------
Q ->  아닙니다.  모독한 적이 없습니다 저는요.
Original A ->  위원장님.
ex A -> 잠깐만요.  그것은 말이지요 한기호 간사님께서 ‘여당 위원들’이라고 했습니다.
------------------------------------------------------------------------------------
Q ->  하늘이 무너져도 도덕은 하늘이 무너져도 정의는 세워야 돼요
Original A ->  이석현 의원
ex A -> 이석현 의원
------------------------------------------------------------------------------------
Q ->  만약 이명박 대통령과 박근혜 새누리당이
Original A ->  자 더 이상 인권위원장후보자에 대한 질의가 없기 때문에 질의를 모두 종결하겠습니다.
ex A -> 자 더 이상 인권위원장후보자에 대한 질의가 없기 때문에 질의를 모두 종결하겠습니다.
------------------------------------------------------------------------------------
Q ->  뭐 약속 있으세요?
Original A ->  협조해 주세요.
ex A -> 